<a href="https://colab.research.google.com/github/pouuh/prospect_bayes/blob/master/read_JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pdb

#通用的读取反射率和温度的Jiso文件

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# clean yield_data
def load_clean_yield_data(yield_data_filepath):
    """
    Cleans the yield data by making sure any Nan values in the columns we care about
    are removed
    """
    important_columns = ['Year', 'State ANSI', 'County ANSI', 'Value']
    yield_data = pd.read_csv(yield_data_filepath).dropna(subset=important_columns,
                                                         how='any')
    return yield_data

#load_clean_yield_data(yield_data_path)[['Year', 'State ANSI', 'County ANSI']].values

In [ ]:
def read_hist_LST(sur_r):
  num_r=len(sur_r)
  num_bands=2
  data_r=np.zeros([num_r,32,num_bands])
  data_r[:]=np.NaN

  sample_count=0

  for feature in sur_r:
    hists=feature['properties']['prop']#['sur_refl_b01']
    bands_count=0
    for bands in hists:
      hist=hists[bands]
      #%debug magic
      #print((bands))
      #print("List: ",len(hist[0]))
      if hist is not None:
        arr = np.array(hist)
        sum_y=sum(arr[:,1]) #normalizing
        data_r[sample_count,:,bands_count]=arr[:,1]/sum_y
      bands_count=bands_count+1
    sample_count=sample_count+1
  return data_r

In [ ]:
def read_hist_Ref(sur_r):
  num_r=len(sur_r)
  num_bands=7
  data_r=np.zeros([num_r,32,num_bands])
  data_r[:]=np.NaN

  sample_count=0

  for feature in sur_r:
    hists=feature['properties']['prop']#['sur_refl_b01']
    bands_count=0
    for bands in hists:
      hist=hists[bands]
      #print((hist))
      #print("List: ",len(hist[0]))
      if hist is not None:
        arr = np.array(hist)
        sum_y=sum(arr[:,1]) #normalizing
        data_r[sample_count,:,bands_count]=arr[:,1]/sum_y
      bands_count=bands_count+1
    sample_count=sample_count+1
  
  return data_r

In [ ]:
# #test
# names='0_01_047'
# data2_ref=pd.read_csv('/content/drive/MyDrive/ImgHist_masked/'+names+'.csv')
# data2_lst=pd.read_csv('/content/drive/MyDrive/ImgLstHist_masked/'+names+'.csv')
# sur_r_ref=data2_ref['prop']
# sur_r_lst=data2_lst['prop']

# #print(shape())
# data_r_lst=read_hist_LST(sur_r_lst)
# data_r_ref=read_hist_Ref(sur_r_ref)
# print(data_r_ref[1][0])

In [ ]:
def read_features(names):
  #读取JSON格式的数据
  import json
  with open('/content/drive/MyDrive/ImgColHist_masked_modis2/Col'+names+'.geojson') as f:
      data2_ref = json.load(f)
  #print(len(data['features']))
  with open('/content/drive/MyDrive/ImgLstHist_masked_modis2/Lst'+names+'.geojson') as f:
      data2_lst = json.load(f)

  data_r_ref=read_hist_Ref(data2_ref['features'])
  data_r_lst=read_hist_LST(data2_lst['features'])

  #合并所有的数据为一个
  data_hist=np.zeros([data_r_lst.shape[0],32,9])
  data_hist[:,:,0:7]=data_r_ref
  data_hist[:,:,7:9]=data_r_lst

  years=13
  #将所有的影像划分为年的单位
  data_hist_np=np.zeros([years,32,32,9])
  for i in range(years):
    idx_start=i*46+6
    idx_end=i*46+38
    #print(data_hist[idx_start:idx_end,:,:].shape)
    data_hist_np[i,:,:,:]=data_hist[idx_start:idx_end,:,:]

  return data_hist_np

In [ ]:
# # #将特征和数据按照同样的顺序存储
# # idx1=idxs_state.get(item)
# # idx2=idxs_county.get(item)
# data_r=np.zeros([10,10])
# data_r[:]=np.NaN
# data_r[1,:]=np.zeros([1,10])
# print(data_r)

In [ ]:
#yield_final=pd.read_csv('/content/drive/MyDrive/earthengine/yield_final_highquality.csv',header=None)
locations = pd.read_csv('/content/drive/MyDrive/earthengine/locations_final.csv',header=None)
yield_data_path='/content/drive/MyDrive/earthengine2/yield_data.csv'
yield_final=load_clean_yield_data(yield_data_path)#[['Year', 'State ANSI', 'County ANSI','Value']].values

#print(yield_final['State ANSI'])

#%debug magic
num_train=0
num_test=0

#samples estimation
train_x=np.zeros([20474,32,32,9])
train_y=np.zeros([20474,1])
train_years=np.zeros([20474,1])

test_x=np.zeros([2000,32,32,9])
test_y=np.zeros([2000,1])

for idx in locations[0].index: 
  #year=yield_final[0].get(idx)-2003
  idx1=locations[0].get(idx)
  idx2=locations[1].get(idx)

  state_str=str(idx1)
  if idx1<10:
    state_str='0'+state_str

  county_str=str(idx2)
  if idx2<100:
    county_str='0'+county_str
    if idx2<10:
      county_str='0'+county_str

  name=str(idx)+'_'+state_str+'_'+county_str

  # data2_ref=pd.read_csv('/content/drive/MyDrive/ImgHist_masked/'+name+'.csv')
  # data2_lst=pd.read_csv('/content/drive/MyDrive/ImgLstHist_masked/'+name+'.csv')
  # sur_r_ref=data2_ref['prop']
  # sur_r_lst=data2_lst['prop']

  # data_r_lst=read_hist_LST(sur_r_lst)
  # data_r_ref=read_hist_Ref(sur_r_ref)

  # %debug magic
  # break

  #读取每个县13年的数据
  input_data=read_features(name)

  #读取该县的历史产量数据
  history_list=yield_final.index[(yield_final['State ANSI'] == idx1) & (yield_final['County ANSI'] == idx2)].tolist()
  #y=yield_final[3].get(idx)
  print(history_list)

  #对该县的每一个年份单独进行读取
  for i in history_list:
    year=yield_final['Year'].get(i)
    #print(year)
    year_f=year-2003
    print(year_f)
    
    if year==2015:#单独存储
      #code
      num_test=num_test+1
      test_x[num_test-1,:,:,:]=input_data[year_f,:,:,:]
      test_y[num_test-1]=yield_final['Value'].get(i)
      continue

    if year==2016:#舍弃2016年的数据
      continue
    
    #print(num_train)
    num_train=num_train+1
    train_x[num_train-1,:,:,:]=input_data[year_f,:,:,:]
    train_y[num_train-1]=yield_final['Value'].get(i)
    train_years[num_train-1]=year

  if num_train > 200:
    break

train_x=train_x[0:num_train]
train_y=train_y[0:num_train]

test_x=test_x[0:num_test]
test_y=test_y[0:num_test]



[0, 1485, 2934, 4509, 6009, 7583, 10726, 18695]
13
12
11
10
9
8
6
1
[2, 1486, 2937, 4511, 7585]
13
12
11
10
8
[3, 1487, 2938, 6010, 18696]
13
12
11
9
1
[6, 1489, 2942, 4515, 6012, 7588, 18698]
13
12
11
10
9
8
1
[7, 1490, 2943, 4516, 6013, 7589, 10730]
13
12
11
10
9
8
6
[8, 1491, 2944, 4517, 6014, 7590, 9127, 10731, 12223, 13652, 15274, 18699, 20454]
13
12
11
10
9
8
7
6
5
4
3
1
0
[10, 1492, 2945, 4519, 6015, 7592, 9129, 10733, 13653, 18700, 20455]
13
12
11
10
9
8
7
6
4
1
0
[11, 1493, 2946, 4520, 6016, 10735, 18701]
13
12
11
10
9
6
1
[13, 1495, 2949, 4522, 6018, 9131]
13
12
11
10
9
7
[14, 1496, 2951, 4524, 6020, 7595, 9133, 10738, 13656, 15277, 16971, 18704, 20458]
13
12
11
10
9
8
7
6
4
3
2
1
0
[15, 1497, 2952, 4525, 6021, 7596, 9134, 10739, 13657, 15278, 16972, 18705, 20459]
13
12
11
10
9
8
7
6
4
3
2
1
0
[16, 1498, 2953, 4526, 6022, 7597, 9135, 10740, 12225, 13658, 15279, 16973, 18706, 20460]
13
12
11
10
9
8
7
6
5
4
3
2
1
0
[17, 1499, 2954, 6023, 7598, 10741, 13659, 15280, 16974, 18707,

In [ ]:
print(train_x[200][3])

In [ ]:
np.save('/content/drive/MyDrive/earthengine2/train_x2',train_x)
np.save('/content/drive/MyDrive/earthengine2/train_y2',train_y)
np.save('/content/drive/MyDrive/earthengine2/test_x2',test_x)
np.save('/content/drive/MyDrive/earthengine2/test_y2',test_y)

In [ ]:
# print(train_x.shape)

In [ ]:
# temp=data_r_ref[0:2,:,:]
# print(temp.shape)